In [1]:
# system packages
import numpy as np
import pandas as pd
import math

#Plotting
import matplotlib.pyplot as mtplt
import matplotlib.gridspec as gridspec
#Jupyter should create an interactive plot wit qt
%matplotlib qt

#Linear Algebraic, signal processing
import scipy.linalg as scLinAlg
import scipy.signal as sigP

#gurobi
import gurobipy as gp
from gurobipy import GRB


In [2]:
# individual packages
import sg, sa, sp, obq

mtplt.close('all')

sNbins = 2**6
sFs = sNbins
sT = 1 / sFs
sL = 83
sL_h = math.floor(sL/2)
sRecFilterFrequ = 3


Generate input signal

In [3]:
### Signal generation ###
v_n = np.arange(sNbins).reshape(-1, 1)
vxFrequ = (np.arange(1, 3, step=3) * sFs / sNbins).reshape(-1, 1)
vxPhaseInit = np.random.rand(len(vxFrequ), 1) * 2 * np.pi

vx, vTime = sg.signalGen(v_n, vxFrequ, vxPhaseInit, sFs, 'real')
vx = sg.MFnormalize(vx, np.array([-1, 1]))

Generate FiltersCoeff and corresp. Matrices

In [4]:
### Generate ideal matrices ###
vRIdeal = sp.idealBinFilt(sNbins, sg.freq2Bin(sRecFilterFrequ, sNbins, sFs), 'normal')
mRIdeal = scLinAlg.toeplitz(vRIdeal)

vFilt = sigP.firwin(sL, sg.freq2digFc(sRecFilterFrequ,sFs), window = "hann")
# Replace zeros with eps
vFilt[vFilt == 0] = np.finfo(float).eps
mFilt = sp.convmtx(vFilt,sNbins,'colWise')
mFiltfull = mFilt[sL_h-1:(sL_h+sNbins-1),:]
mFilt = np.tril(mFiltfull)

Quantize the input signal

In [5]:
vC = np.zeros((sNbins,1), dtype = 'float')
vBSequSingle, ve = obq.iterSequQ(vx,mFilt,vC,0)
vBSequBlock, vEBlock = obq.iterSequQBlock(vx,mFiltfull,vC)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-05
Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 0 rows, 64 columns and 0 nonzeros
Model fingerprint: 0xdcfb44eb
Model has 2080 quadratic objective terms
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e-01, 8e+00]
  QObjective range [1e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 95.1136922
Found heuristic solution: objective 86.4523365
Found heuristic solution: objective 0.0674239
Found heuristic solution: objective 0.0471290
Found heuristic solution: objective 0.0403497
Found heuristic solut

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\mayerflo\AppData\Roaming\Python\Python312\site-packages\ipykernel\iostream.py", line 655, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 27052916 5253553     cutoff   71         0.03407   -0.02339   169%   2.9  930s


Frequency Analysis

In [ ]:
vBfft = np.fft.fft(vBSequSingle)
vBfftMag = 20*sa.safelog10(np.abs(vBfft) / np.max(abs(vBfft))) 

vBBlockfft = np.fft.fft(vBSequBlock)
vBBlockfftMag = 20*sa.safelog10(np.abs(vBBlockfft) / np.max(abs(vBBlockfft))) 

vX = np.fft.fft(vx)
vXMag = 20*sa.safelog10(np.abs(vX) / np.max(abs(vX)))
# Frequency bins
vFreq = np.fft.fftfreq(sNbins, sT)

SNR Calculations

In [ ]:
sVX_MSE, sVX_SNRdb, sVX_PSNRdb = sa.evalN(mRIdeal @ (vx-vx), mRIdeal @ vx)
sVB_MSE, sVB_SNRdb, sVB_PSNRdb = sa.evalN(mRIdeal @ (vx-vBSequSingle), mRIdeal @ vx)
sVBBlock_MSE, sVBBlock_SNRdb, sVBBlock_PSNRdb = sa.evalN(mRIdeal @ (vx-vBSequBlock), mRIdeal @ vx)

Plots

In [ ]:
###### PLOTTING ######
figOne = mtplt.figure()
Pltgs = gridspec.GridSpec(3, 2)

pltDiscTime = figOne.add_subplot(Pltgs[0,:])
pltDiscTime.plot(vx)
pltDiscTime.set_title('Input Signal SNR: {snr} dB'.format(snr = round(sVX_SNRdb,2)))
pltDiscTime.set_xlabel('Samples $n$', fontsize = 11)
pltDiscTime.set_ylabel('Amplitude', fontsize = 11)
pltDiscTime.set_xlim([0,sNbins])
mtplt.minorticks_on()
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

pltObsOne = figOne.add_subplot(Pltgs[1,0])
pltObsOne.plot(vBSequSingle)
pltObsOne.set_title('Single-Sequential Quantization')
pltObsOne.set_xlabel('Samples $n$', fontsize = 11)
pltObsOne.set_ylabel('Amplitude', fontsize = 11)
pltObsOne.set_xlim([0,sNbins])
mtplt.minorticks_on()
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

pltObsTwo = figOne.add_subplot(Pltgs[1,1])
pltObsTwo.plot(vBSequBlock)
pltObsTwo.set_title('Block-Sequential Structure')
pltObsTwo.set_xlabel('Samples $n$', fontsize = 11)
pltObsTwo.set_ylabel('Amplitude', fontsize = 11)
mtplt.minorticks_on()
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

pltFreqOne = figOne.add_subplot(Pltgs[2,0])
pltFreqOne.plot(vFreq[:sNbins // 2], vBfftMag[:sNbins // 2])
pltFreqOne.set_title('Frequency Spectrum One SNR: {snr} dB'.format(snr = round(sVB_SNRdb,2)))
pltFreqOne.set_xlabel('Samples $n$', fontsize = 11)
pltFreqOne.set_ylabel('Magnitude $(dB)$', fontsize = 11)
pltFreqOne.set_xlim([0,sNbins/2])
pltFreqOne.set_ylim([-60,5])
mtplt.minorticks_on()
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

pltFreqTwo = figOne.add_subplot(Pltgs[2,1])
pltFreqTwo.plot(vFreq[:sNbins // 2], vBBlockfftMag[:sNbins // 2])
pltFreqTwo.set_title('Frequency Spectrum Two SNR: {snr} dB'.format(snr = round(sVBBlock_SNRdb,2)))
pltFreqTwo.set_xlabel('Samples $n$', fontsize = 11)
pltFreqTwo.set_ylabel('Magnitude $(dB)$', fontsize = 11)
mtplt.minorticks_on()
pltFreqTwo.set_xlim([0,sNbins/2])
pltFreqTwo.set_ylim([-60,5])
mtplt.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray')

mtplt.tight_layout(pad=-0.25)
mtplt.show()